In [1]:
import numpy as np
import pyopencl as cl
import pyopencl.array

In [44]:
%load_ext pyopencl.ipython_ext

The pyopencl.ipython_ext extension is already loaded. To reload it, use:
  %reload_ext pyopencl.ipython_ext


In [45]:
ctx = cl.create_some_context(interactive=True)

In [46]:
queue = cl.CommandQueue(ctx)

In [47]:
%%cl_kernel -o "-cl-fast-relaxed-math"

__kernel void add_vect(__global const float *a,__global const float *b, __global float *c)
{
    int gid = get_global_id(0);
    c[gid] = a[gid] + b[gid];
}

In [51]:
n = 10000

a = cl.array.empty(queue, n, dtype=np.float32)
a.fill(15)

b_host = np.random.randn(n).astype(np.float32)
b = cl.array.to_device(queue,b_host)

c = cl.array.empty_like(a)

In [53]:
add_vect(queue, (n,), None, a.data, b.data, c.data)

In [55]:
from pyopencl.elementwise import ElementwiseKernel

In [56]:
scale = ElementwiseKernel(ctx,'float c, float *x','x[i] = c * x[i]')

In [62]:
scale_add = ElementwiseKernel(ctx,'float c, float *x, float *y','y[i] = c * x[i] + y[i]')

In [63]:
scale_add(5,a,b)

In [64]:
b.get()

array([ 73.71160126,  65.83309937,  72.35173035, ...,  67.85299683,
        86.69250488,  62.55369949], dtype=float32)

In [70]:
dev = cl.get_platforms()[0].get_devices()[1]

In [75]:
cl.characterize.usable_local_mem_size(dev)

65536

In [85]:
dev.

512